In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from bisect import bisect_left, bisect_right

class SortedCollection(object):
    '''Sequence sorted by a key function.

    SortedCollection() is much easier to work with than using bisect() directly.
    It supports key functions like those use in sorted(), min(), and max().
    The result of the key function call is saved so that keys can be searched
    efficiently.

    Instead of returning an insertion-point which can be hard to interpret, the
    five find-methods return a specific item in the sequence. They can scan for
    exact matches, the last item less-than-or-equal to a key, or the first item
    greater-than-or-equal to a key.

    Once found, an item's ordinal position can be located with the index() method.
    New items can be added with the insert() and insert_right() methods.
    Old items can be deleted with the remove() method.

    The usual sequence methods are provided to support indexing, slicing,
    length lookup, clearing, copying, forward and reverse iteration, contains
    checking, item counts, item removal, and a nice looking repr.

    Finding and indexing are O(log n) operations while iteration and insertion
    are O(n).  The initial sort is O(n log n).

    The key function is stored in the 'key' attibute for easy introspection or
    so that you can assign a new key function (triggering an automatic re-sort).

    In short, the class was designed to handle all of the common use cases for
    bisect but with a simpler API and support for key functions.

    >>> from pprint import pprint
    >>> from operator import itemgetter

    >>> s = SortedCollection(key=itemgetter(2))
    >>> for record in [
    ...         ('roger', 'young', 30),
    ...         ('angela', 'jones', 28),
    ...         ('bill', 'smith', 22),
    ...         ('david', 'thomas', 32)]:
    ...     s.insert(record)

    >>> pprint(list(s))         # show records sorted by age
    [('bill', 'smith', 22),
     ('angela', 'jones', 28),
     ('roger', 'young', 30),
     ('david', 'thomas', 32)]

    >>> s.find_le(29)           # find oldest person aged 29 or younger
    ('angela', 'jones', 28)
    >>> s.find_lt(28)           # find oldest person under 28
    ('bill', 'smith', 22)
    >>> s.find_gt(28)           # find youngest person over 28
    ('roger', 'young', 30)

    >>> r = s.find_ge(32)       # find youngest person aged 32 or older
    >>> s.index(r)              # get the index of their record
    3
    >>> s[3]                    # fetch the record at that index
    ('david', 'thomas', 32)

    >>> s.key = itemgetter(0)   # now sort by first name
    >>> pprint(list(s))
    [('angela', 'jones', 28),
     ('bill', 'smith', 22),
     ('david', 'thomas', 32),
     ('roger', 'young', 30)]

    '''

    def __init__(self, iterable=(), key=None):
        self._given_key = key
        key = (lambda x: x) if key is None else key
        decorated = sorted((key(item), item) for item in iterable)
        self._keys = [k for k, item in decorated]
        self._items = [item for k, item in decorated]
        self._key = key

    def _getkey(self):
        return self._key

    def _setkey(self, key):
        if key is not self._key:
            self.__init__(self._items, key=key)

    def _delkey(self):
        self._setkey(None)

    key = property(_getkey, _setkey, _delkey, 'key function')

    def clear(self):
        self.__init__([], self._key)

    def copy(self):
        return self.__class__(self, self._key)

    def __len__(self):
        return len(self._items)

    def __getitem__(self, i):
        return self._items[i]

    def __iter__(self):
        return iter(self._items)

    def __reversed__(self):
        return reversed(self._items)

    def __repr__(self):
        return '%s(%r, key=%s)' % (
            self.__class__.__name__,
            self._items,
            getattr(self._given_key, '__name__', repr(self._given_key))
        )

    def __reduce__(self):
        return self.__class__, (self._items, self._given_key)

    def __contains__(self, item):
        k = self._key(item)
        i = bisect_left(self._keys, k)
        j = bisect_right(self._keys, k)
        return item in self._items[i:j]

    def index(self, item):
        'Find the position of an item.  Raise ValueError if not found.'
        k = self._key(item)
        i = bisect_left(self._keys, k)
        j = bisect_right(self._keys, k)
        return self._items[i:j].index(item) + i

    def count(self, item):
        'Return number of occurrences of item'
        k = self._key(item)
        i = bisect_left(self._keys, k)
        j = bisect_right(self._keys, k)
        return self._items[i:j].count(item)

    def insert(self, item):
        'Insert a new item.  If equal keys are found, add to the left'
        k = self._key(item)
        i = bisect_left(self._keys, k)
        self._keys.insert(i, k)
        self._items.insert(i, item)

    def insert_right(self, item):
        'Insert a new item.  If equal keys are found, add to the right'
        k = self._key(item)
        i = bisect_right(self._keys, k)
        self._keys.insert(i, k)
        self._items.insert(i, item)

    def remove(self, item):
        'Remove first occurence of item.  Raise ValueError if not found'
        i = self.index(item)
        del self._keys[i]
        del self._items[i]

    def find(self, k):
        'Return first item with a key == k.  Raise ValueError if not found.'
        i = bisect_left(self._keys, k)
        if i != len(self) and self._keys[i] == k:
            return self._items[i]
        raise ValueError('No item found with key equal to: %r' % (k,))

    def find_le(self, k):
        'Return last item with a key <= k.  Raise ValueError if not found.'
        i = bisect_right(self._keys, k)
        if i:
            return self._items[i-1]
        raise ValueError('No item found with key at or below: %r' % (k,))

    def find_lt(self, k):
        'Return last item with a key < k.  Raise ValueError if not found.'
        i = bisect_left(self._keys, k)
        if i:
            return self._items[i-1]
        raise ValueError('No item found with key below: %r' % (k,))

    def find_ge(self, k):
        'Return first item with a key >= equal to k.  Raise ValueError if not found'
        i = bisect_left(self._keys, k)
        if i != len(self):
            return self._items[i]
        raise ValueError('No item found with key at or above: %r' % (k,))

    def find_gt(self, k):
        'Return first item with a key > k.  Raise ValueError if not found'
        i = bisect_right(self._keys, k)
        if i != len(self):
            return self._items[i]
        raise ValueError('No item found with key above: %r' % (k,))

    def argfind(self, k):
        'ADDED: Return the index of the item with a key == k.  Raise ValueError if not found.'
        try:
            r = self.find(k)
            return self.index(r)
        except Exception as e:
            raise e

    def pop_first(self):
        i = 0
        item = self[i]
        del self._keys[i]
        del self._items[i]
        return item


from collections import namedtuple

# Implement P-cell as a namedtuple with properties pos (in P-space), 
# wait (division waiting time), lineage
Pcell = namedtuple('Pcell', ['pos', 'wait', 'fitness', 'lineage'])

# Implement D-cell as a namedtiple with properties pos (in D-space),
# wait (loss waiting time), lineage
Dcell = namedtuple('Dcell', ['pos', 'wait', 'fitness', 'lineage'])

# Event queue contains either DivisionEvent or LossEvent
DivisionEvent = namedtuple('DivisionEvent', ['pos', 'wait', 'lineage'])
LossEvent = namedtuple('LossEvent', ['pos', 'wait', 'lineage'])

In [3]:
# Parameters

# P-cell population size
Np = int(1e4)

# Rate of progenitor cell division
division_rate = 1.
division_rate_r = np.reciprocal(division_rate)
# Rate of differentiating cell loss
# Reciprocal loss_rate_r is the mean lifetime of D-cells
loss_rate = 1.
loss_rate_r = np.reciprocal(loss_rate)

# progenitor and differentiation probabilities
pp_prob = 0.1
pd_prob = 0.8
dd_prob = 0.1

# Wildtype lineage id
default_lineage_id = 0
# Wildtype fitness
default_fitness = 1.

# RNG seed
seed = 64745

In [4]:
# Create a 10000 linear lattice using a dictionary
# then reduce every 1000 blocks into a single AF to get a 10 samples
Np = 10

In [5]:
# TEST
# Every pos is a lineage

# Init a pseudo-random generator
rng = np.random.default_rng(seed)

from operator import itemgetter
from collections import defaultdict

# Init waiting times
# Init queue
# Use SortedCollection to create a FIFO event queue tracking division and loss
event_queue = SortedCollection(key=itemgetter(1))

# Init lineage tracking
existing_lineages = defaultdict(int)

# Init space
p_space = []
for pos, wait in enumerate(rng.exponential(division_rate_r, size=Np)):
    p_cell = Pcell(pos, wait, default_fitness, pos)
    p_space.append(p_cell)
    event_queue.insert(DivisionEvent(p_cell.pos, p_cell.wait, p_cell.lineage))
    existing_lineages[pos] = 1 

print(len(p_space), len(event_queue), '\n')

current_time = 0
print([i.lineage for i in p_space], current_time)
while len(existing_lineages) > 1:
# for _ in range(50):
    # Pop current event
    current_event = event_queue.pop_first()
    current_time = current_event.wait
    target = p_space[current_event.pos]

    # Given the current cell, pick a neighbor
    neighbor_pos = target.pos + rng.choice([-1, 1])
    if neighbor_pos >= Np:
        neighbor_pos = neighbor_pos - Np
    neighbor = p_space[neighbor_pos]

    # Compare current cell's and neighbor's fitnesses
    threshold = (target.fitness - neighbor.fitness) + 0.5
    if rng.random() < threshold:
        # New cells
        new_neighbor = Pcell(
            neighbor.pos, 
            current_time + rng.exponential(division_rate_r),
            target.fitness,
            target.lineage
        )
        p_space[neighbor.pos] = new_neighbor
        # Add to queue
        event_queue.insert(DivisionEvent(new_neighbor.pos, new_neighbor.wait, new_neighbor.lineage))

        # update lineage tracking
        existing_lineages[neighbor.lineage] -= 1
        existing_lineages[new_neighbor.lineage] += 1
        if existing_lineages[neighbor.lineage] == 0:
            del existing_lineages[neighbor.lineage]

        print(target.pos, '->', neighbor.pos)
        print([i.lineage for i in p_space], current_time, list(existing_lineages.keys()))
    # If more than threshold nothing happens
    new_target = Pcell(
        target.pos, 
        current_time + rng.exponential(division_rate_r),
        target.fitness,
        target.lineage
    )
    p_space[target.pos] = new_target
    event_queue.insert(DivisionEvent(new_target.pos, new_target.wait, new_target.lineage))



10 10 

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] 0
8 -> 7
[0, 1, 2, 3, 4, 5, 6, 8, 8, 9] 0.525190404720584 [0, 1, 2, 3, 4, 5, 6, 8, 9]
3 -> 4
[0, 1, 2, 3, 3, 5, 6, 8, 8, 9] 0.5733483635067674 [0, 1, 2, 3, 5, 6, 8, 9]
3 -> 2
[0, 1, 3, 3, 3, 5, 6, 8, 8, 9] 0.6329948188784735 [0, 1, 3, 5, 6, 8, 9]
1 -> 0
[1, 1, 3, 3, 3, 5, 6, 8, 8, 9] 0.7084644783974737 [1, 3, 5, 6, 8, 9]
4 -> 5
[1, 1, 3, 3, 3, 3, 6, 8, 8, 9] 0.8432622806577952 [1, 3, 6, 8, 9]
2 -> 3
[1, 1, 3, 3, 3, 3, 6, 8, 8, 9] 0.9383504527982302 [1, 3, 6, 8, 9]
4 -> 5
[1, 1, 3, 3, 3, 3, 6, 8, 8, 9] 1.0806355581604004 [1, 3, 6, 8, 9]
0 -> 9
[1, 1, 3, 3, 3, 3, 6, 8, 8, 1] 1.0914930451320786 [1, 3, 6, 8]
7 -> 6
[1, 1, 3, 3, 3, 3, 8, 8, 8, 1] 1.1204115775763241 [1, 3, 8]
0 -> 9
[1, 1, 3, 3, 3, 3, 8, 8, 8, 1] 1.1727746725402521 [1, 3, 8]
2 -> 3
[1, 1, 3, 3, 3, 3, 8, 8, 8, 1] 1.2275064393795196 [1, 3, 8]
4 -> 5
[1, 1, 3, 3, 3, 3, 8, 8, 8, 1] 1.296538883582523 [1, 3, 8]
5 -> 4
[1, 1, 3, 3, 3, 3, 8, 8, 8, 1] 1.4214366521490118 [1, 3, 8]
2 -> 3
[1, 1, 3

In [6]:
# Event queue adds a third event - MutationEvent
MutationEvent = namedtuple('MutationEvent', ['pos', 'wait', 'lineage'])

In [7]:
# 2-lineage sim, equal replacement probabilities

seed = np.random.randint(int(1e9))
print(seed)
# Init a pseudo-random generator
rng = np.random.default_rng(seed)

from operator import itemgetter
from collections import defaultdict

# Lineages
wildtype_lineage = '0'
subclone1_lineage = None
subclone2_lineage = None
next_lineage_id = 1

# Fitness as a probability of replacing [0, 1.0]
wildtype_fitness = 0.5
subclone1_fitness = 0.5
subclone2_fitness = 0.5

# subclone 1 - 2 waiting time mean
betweeen_subclone_waiting_time_mean = 2.0

# Init waiting times
# Init queue
# Use SortedCollection to create a FIFO event queue tracking division and loss
event_queue = SortedCollection(key=itemgetter(1))

# Init lineage tracking
existing_lineages = defaultdict(int)

# Init space with all wildtype
p_space = []
# Introduce subclone 1 randomly in space
pos_choices = list(range(Np))
subclone1_pos = rng.choice(pos_choices)

for pos, wait in enumerate(rng.exponential(division_rate_r, size=Np)):
    if pos == subclone1_pos:
        subclone1_lineage = wildtype_lineage + str(next_lineage_id)
        next_lineage_id += 1
        p_cell = Pcell(pos, wait, subclone1_fitness, subclone1_lineage)
    else:
        p_cell = Pcell(pos, wait, wildtype_fitness, wildtype_lineage)
    p_space.append(p_cell)
    event_queue.insert(DivisionEvent(p_cell.pos, p_cell.wait, p_cell.lineage))
    existing_lineages[p_cell.lineage] += 1

# Pick a waiting time to introduce subclone 2
subclone2_wait = rng.exponential(betweeen_subclone_waiting_time_mean)
subclone2_pos = rng.choice(pos_choices)
event_queue.insert(MutationEvent(subclone2_pos, subclone2_wait, subclone2_lineage))

print(len(p_space), len(event_queue), list(existing_lineages.keys()))
print('subclone1 ->', subclone1_pos)
print([i.lineage for i in p_space], current_time, list(existing_lineages.keys()))
print('subclone 2 will be introduced after', subclone2_wait, '\n')

current_time = 0
print([i.lineage for i in p_space], current_time)
while len(existing_lineages) > 1:
# for _ in range(50):
    # Pop current event
    current_event = event_queue.pop_first()
    current_time = current_event.wait
    target = p_space[current_event.pos]
    
    # Mutation event only
    if isinstance(current_event, MutationEvent):
        # Remove events associated with old neighbor from the event queue
        event_queue.remove(DivisionEvent(target.pos, target.wait, target.lineage))
        subclone2_lineage = target.lineage + str(next_lineage_id)
        next_lineage_id += 1
        subclone2 = Pcell(
            target.pos, 
            current_time + rng.exponential(division_rate_r),
            subclone2_fitness,
            subclone2_lineage
        )
        p_space[target.pos] = subclone2
        # Add to queue
        event_queue.insert(DivisionEvent(subclone2.pos, subclone2.wait, subclone2.lineage))
        # update lineage tracking
        existing_lineages[target.lineage] -= 1
        existing_lineages[subclone2_lineage] += 1
        if existing_lineages[target.lineage] == 0:
            del existing_lineages[target.lineage]
        
        print('subclone2 ->', target.pos)
        print([i.lineage for i in p_space], current_time, list(existing_lineages.keys()))
        continue
        
    # Given the current cell, pick a neighbor
    neighbor_pos = target.pos + rng.choice([-1, 1])
    if neighbor_pos >= Np:
        neighbor_pos = neighbor_pos - Np
    neighbor = p_space[neighbor_pos]

    # Compare current cell's and neighbor's fitnesses
    threshold = (target.fitness - neighbor.fitness) + 0.5
    if rng.random() < threshold:
        print(target.pos, neighbor.pos)
        # Remove events associated with old neighbor from the event queue
        event_queue.remove(DivisionEvent(neighbor.pos, neighbor.wait, neighbor.lineage))
        # New cell
        new_neighbor = Pcell(
            neighbor.pos, 
            current_time + rng.exponential(division_rate_r),
            target.fitness,
            target.lineage
        )
        p_space[neighbor.pos] = new_neighbor
        # Add to queue
        event_queue.insert(DivisionEvent(new_neighbor.pos, new_neighbor.wait, new_neighbor.lineage))

        # update lineage tracking
        existing_lineages[neighbor.lineage] -= 1
        existing_lineages[new_neighbor.lineage] += 1
        if existing_lineages[neighbor.lineage] == 0:
            del existing_lineages[neighbor.lineage]

        print(target.pos, '->', neighbor.pos)
        print([i.lineage for i in p_space], current_time, list(existing_lineages.keys()))

        # Stop if subclone fixes or is lost
        if current_time > subclone2_wait and len(existing_lineages) < 3:
            print('STOP')
            break

    # If more than threshold nothing happens
    new_target = Pcell(
        target.pos, 
        current_time + rng.exponential(division_rate_r),
        target.fitness,
        target.lineage
    )
    p_space[target.pos] = new_target
    event_queue.insert(DivisionEvent(new_target.pos, new_target.wait, new_target.lineage))

# put in a separate script
# ccsim/2_subclone_sim.py


182601533
10 11 ['0', '01']
subclone1 -> 4
['0', '0', '0', '0', '01', '0', '0', '0', '0', '0'] 5.576765734305676 ['0', '01']
subclone 2 will be introduced after 0.3712650054517677 

['0', '0', '0', '0', '01', '0', '0', '0', '0', '0'] 0
9 0
9 -> 0
['0', '0', '0', '0', '01', '0', '0', '0', '0', '0'] 0.10640099731980704 ['0', '01']
3 2
3 -> 2
['0', '0', '0', '0', '01', '0', '0', '0', '0', '0'] 0.33010037069698883 ['0', '01']
subclone2 -> 1
['0', '02', '0', '0', '01', '0', '0', '0', '0', '0'] 0.3712650054517677 ['0', '01', '02']
2 3
2 -> 3
['0', '02', '0', '0', '01', '0', '0', '0', '0', '0'] 0.4410128170410761 ['0', '01', '02']
6 7
6 -> 7
['0', '02', '0', '0', '01', '0', '0', '0', '0', '0'] 1.4315970886812204 ['0', '01', '02']
0 9
0 -> 9
['0', '02', '0', '0', '01', '0', '0', '0', '0', '0'] 1.670697326891612 ['0', '01', '02']
4 5
4 -> 5
['0', '02', '0', '0', '01', '01', '0', '0', '0', '0'] 1.8292178641759755 ['0', '01', '02']
7 6
7 -> 6
['0', '02', '0', '0', '01', '01', '0', '0', '0', '0'] 